In [1]:
# %%html
# <style>div.input{display:none} div.output_stderr{display:none}</style>

[KnowEnG Signature_Analysis_Pipeline on Github](https://github.com/KnowEnG-Research/Signature_Analysis_Pipeline)

# KnowEnG Signiture Analysis Notebook - Development 11 / 14 / 2017 
* Context: https://github.com/dlanier/notebooks_KnowEnG.git
* Running on notebooks.knoweng.org server
* Where individual user notebooks share KnowEnG_*_Pipelines/src

### Develop Here: Notebook widgets to set run parameters for Signiture Analysis Pipeline
* Completed
    * function to set run_parameters to local directories from a dict.  (set_local_run_parameters)
    * define all run parameters. (run_parameters_df = get_yaml_df(YAML_DIR))
    
* Goal: function(s) display run_parameters selectors and a run button callback function.
    * Input Impossibilities:
        * method choice defines file keys requirement -- gray out impossible choices
        * method choice defines options possiblilities -- gray out impossible options

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import io
import sys
import base64

import pandas as pd
from pandas.io.common import EmptyDataError
import numpy as np

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import traitlets

from knpackage import toolbox as kn

sys.path.insert(1, '../Signature_Analysis_Pipeline/src')
import gene_signature_toolbox as gst

sys.path.insert(1, '../notebooks_KnowEnG/src')
from layout_notebooks import *
from localize_run_parameters import *

#       These should be imported from a local module:
SPREADSHEETS_DIR = os.path.abspath('../Signature_Analysis_Pipeline/data/spreadsheets')
NETWORKS_DIR = os.path.abspath('../Signature_Analysis_Pipeline/data/spreadsheets')
YAML_DIR = os.path.abspath('../Signature_Analysis_Pipeline/data/run_files')

results_dir = USER_RESULTS_DIRECTORY
input_data_dir = USER_DATA_DIRECTORY
clear_output()

#       Create a dictionary to translate BENCHMARK yaml files to local directory names:
#       Tested on (jupyter notebook) laptop, desktop and (Jupyterhub) server
run_dir = os.getcwd()
local_dict = {'data/spreadsheets': SPREADSHEETS_DIR, 
              'data/networks': NETWORKS_DIR, 
              'run_dir' : run_dir, 
              'data/run_files' : YAML_DIR}

def set_local_run_parameters(run_parameters, local_dict):
    """ run_parameters = set_local_run_parameters(run_parameters, local_dict) 
        (as implemented in : notebooks_KnowEnG/src/localize_run_parameters.py)
    """
    for key_name, key_value in local_dict.items():
        for k, v in run_parameters.items():
            if 'full_path' in k or 'directory' in k:
                if key_name in v:
                    de_nada, f_name = os.path.split(v)
                    run_parameters[k] = os.path.join(key_value, f_name)
                
    return run_parameters

In [3]:
# yaml_files = get_yaml_files_list(YAML_DIR)
# display_all_yaml_files(YAML_DIR, yaml_files)
display_yaml_files_list(YAML_DIR)

/Users/lanier4/git_clone/dlanier/Signature_Analysis_Pipeline/data/run_files
	 BENCHMARK_1_GS_cos.yml
	 BENCHMARK_1_GS_spearman.yml
	 BENCHMARK_2_GS_net_cos.yml
	 BENCHMARK_2_GS_net_spearman.yml
	 BENCHMARK_3_GS_cc_cos.yml
	 BENCHMARK_3_GS_cc_spearman.yml
	 BENCHMARK_4_GS_cc_net_cos.yml
	 BENCHMARK_4_GS_cc_net_spearman.yml
	 TEST_1_GS_cos.yml
	 TEST_1_GS_spearman.yml
	 TEST_2_GS_net_cos.yml
	 TEST_2_GS_net_spearman.yml
	 TEST_3_GS_cc_cos.yml
	 TEST_3_GS_cc_spearman.yml
	 TEST_4_GS_cc_net_cos.yml
	 TEST_4_GS_cc_net_spearman.yml
	 zTEMPLATE_cc_net_cos.yml
	 zTEMPLATE_cc_net_spearman.yml


In [4]:
run_file_name = 'TEST_1_GS_cos.yml'
run_parameters = kn.get_run_parameters(YAML_DIR, run_file_name)
print('\t%s\n'%run_file_name)
for k, v in run_parameters.items():
    print('%30s : %s'%(k,v))

print('\nAvailable Directories:')
for d in os.listdir():
    if os.path.isdir(d) and d[0] != '.':
        print(d)

	TEST_1_GS_cos.yml

    spreadsheet_name_full_path : ../data/spreadsheets/TEST_1_gene_sample.tsv
                 run_directory : /Users/lanier4/git_clone/dlanier/Signature_Analysis_Pipeline/data/run_files
             results_directory : ./run_dir/results
                      run_file : TEST_1_GS_cos.yml
            similarity_measure : cosine
      signature_name_full_path : ../data/spreadsheets/TEST_1_signature.tsv
                        method : similarity
                 tmp_directory : ./run_dir/tmp

Available Directories:
build
data
results
src
test
user_data


In [5]:
run_parameters_lcl = set_local_run_parameters(run_parameters, local_dict)
for k, v in run_parameters_lcl.items():
    print('%30s : %s'%(k,v))

    spreadsheet_name_full_path : /Users/lanier4/git_clone/dlanier/Signature_Analysis_Pipeline/data/spreadsheets/TEST_1_gene_sample.tsv
                 run_directory : /Users/lanier4/git_clone/dlanier/Signature_Analysis_Pipeline/data/run_files/run_files
             results_directory : /Users/lanier4/git_clone/dlanier/notebooks_KnowEnG/results
                      run_file : TEST_1_GS_cos.yml
            similarity_measure : cosine
      signature_name_full_path : /Users/lanier4/git_clone/dlanier/Signature_Analysis_Pipeline/data/spreadsheets/TEST_1_signature.tsv
                        method : similarity
                 tmp_directory : /Users/lanier4/git_clone/dlanier/notebooks_KnowEnG/tmp


In [6]:
#                    This demonstrates that the pipeline runs and writes to results:
gst.run_similarity(run_parameters_lcl)

## Run Parameters Summary DataFrame
* Each column is an available yaml file
* Each row is a possible input (widget)
* unused inputs are filled with "not_used"

In [7]:
run_parameters_df = get_yaml_df(YAML_DIR)
run_parameters_df

,BENCHMARK_1_GS_cos.yml,BENCHMARK_1_GS_spearman.yml,BENCHMARK_2_GS_net_cos.yml,BENCHMARK_2_GS_net_spearman.yml,BENCHMARK_3_GS_cc_cos.yml,BENCHMARK_3_GS_cc_spearman.yml,BENCHMARK_4_GS_cc_net_cos.yml,BENCHMARK_4_GS_cc_net_spearman.yml,TEST_1_GS_cos.yml,TEST_1_GS_spearman.yml,TEST_2_GS_net_cos.yml,TEST_2_GS_net_spearman.yml,TEST_3_GS_cc_cos.yml,TEST_3_GS_cc_spearman.yml,TEST_4_GS_cc_net_cos.yml,TEST_4_GS_cc_net_spearman.yml,zTEMPLATE_cc_net_cos.yml,zTEMPLATE_cc_net_spearman.yml
spreadsheet_name_full_path,../data/spreadsheets/Hsap.nbs_OV.G.gene_som_mu...,../data/spreadsheets/Jinfeng_spreadsheet_sampl...,../data/spreadsheets/Hsap.nbs_OV.G.gene_som_mu...,../data/spreadsheets/Jinfeng_spreadsheet_sampl...,../data/spreadsheets/Hsap.nbs_OV.G.gene_som_mu...,../data/spreadsheets/Jinfeng_spreadsheet_sampl...,../data/spreadsheets/Hsap.nbs_OV.G.gene_som_mu...,../data/spreadsheets/Jinfeng_spreadsheet_sampl...,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/Hsap.nbs_OV.G.gene_som_mu...
cluster_shared_volumn,not used,not used,not used,not used,none,none,none,none,not used,not used,not used,not used,none,none,none,none,none,none
results_directory,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results
similarity_measure,cosine,spearman,cosine,spearman,cosine,spearman,cosine,spearman,cosine,spearman,cosine,spearman,cosine,spearman,cosine,spearman,cosine,spearman
signature_name_full_path,../data/spreadsheets/Hsap.nbs_UCEC.G.gene_som_...,../data/spreadsheets/Jinfeng_spreadsheet_signa...,../data/spreadsheets/Hsap.nbs_UCEC.G.gene_som_...,../data/spreadsheets/Jinfeng_spreadsheet_signa...,../data/spreadsheets/Hsap.nbs_UCEC.G.gene_som_...,../data/spreadsheets/Jinfeng_spreadsheet_signa...,../data/spreadsheets/Hsap.nbs_UCEC.G.gene_som_...,../data/spreadsheets/Jinfeng_spreadsheet_signa...,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/Hsap.nbs_UCEC.G.gene_som_...
rows_sampling_fraction,not used,not used,not used,not used,0.8,0.7,0.8,0.7,not used,not used,not used,not used,0.8,0.8,0.8,0.8,0.8,0.8
rwr_restart_probability,not used,not used,0.7,0.7,0.7,0.7,0.7,0.7,not used,not used,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7
run_directory,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlani

## Run Parameters keys dictionary:
* Completed
    * files list, key types, methods list
    
* Wish list:
    * widget controls need limits as well as types for non-path keys

In [8]:
key_dict, methods_list, yaml_files_list = get_parameter_keys_dict(YAML_DIR)

print('%40s :'%'Yaml Files Included')
for yf in yaml_files_list:
    print('%40s'%yf)

print('\n%40s : %s'%('All yaml files - Keys','types'))
for k, v in key_dict.items():
    print('%40s : %s'%(k,v))
    
print('\n%40s'%'All Available Methods:')
for m in methods_list:
    print('%40s'%m)

                     Yaml Files Included :
                  BENCHMARK_1_GS_cos.yml
             BENCHMARK_1_GS_spearman.yml
              BENCHMARK_2_GS_net_cos.yml
         BENCHMARK_2_GS_net_spearman.yml
               BENCHMARK_3_GS_cc_cos.yml
          BENCHMARK_3_GS_cc_spearman.yml
           BENCHMARK_4_GS_cc_net_cos.yml
      BENCHMARK_4_GS_cc_net_spearman.yml
                       TEST_1_GS_cos.yml
                  TEST_1_GS_spearman.yml
                   TEST_2_GS_net_cos.yml
              TEST_2_GS_net_spearman.yml
                    TEST_3_GS_cc_cos.yml
               TEST_3_GS_cc_spearman.yml
                TEST_4_GS_cc_net_cos.yml
           TEST_4_GS_cc_net_spearman.yml
                zTEMPLATE_cc_net_cos.yml
           zTEMPLATE_cc_net_spearman.yml

                   All yaml files - Keys : types
              spreadsheet_name_full_path : string
                   cluster_shared_volumn : string
                       results_directory : string
                   

## Run Parameters Method DataFrame


In [61]:
def get_key_value_df_II(par_df, key_name, key_value, drop_not_used=True):
    """ get the dataframe where row 'key_name' values match 'key_value' """
    col_dict = (par_df.loc[key_name] == key_value).to_dict()
    col_list = []
    for k, v in col_dict.items():
        if v == True:
            col_list.append(k)

    if len(col_list) > 0:
        par_df = pd.DataFrame(par_df[col_list])
        
    rows_used_list = []
    for row_name in par_df.index:
        if (par_df.loc[row_name].values == 'not used').any() == False:
            rows_used_list.append(row_name)
    if len(rows_used_list) > 0:
        par_df = pd.DataFrame(par_df.loc[rows_used_list])
    
    return par_df

In [60]:
par_df = run_parameters_df
key_name = 'method'
key_value = 'similarity'
similarity_df = get_key_value_df(par_df, key_name, key_value)
similarity_df

,BENCHMARK_1_GS_cos.yml,TEST_1_GS_cos.yml,BENCHMARK_1_GS_spearman.yml,TEST_1_GS_spearman.yml
spreadsheet_name_full_path,../data/spreadsheets/Hsap.nbs_OV.G.gene_som_mu...,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/Jinfeng_spreadsheet_sampl...,../data/spreadsheets/TEST_1_gene_sample.tsv
results_directory,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results
similarity_measure,cosine,cosine,spearman,spearman
signature_name_full_path,../data/spreadsheets/Hsap.nbs_UCEC.G.gene_som_...,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/Jinfeng_spreadsheet_signa...,../data/spreadsheets/TEST_1_signature.tsv
run_directory,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...
tmp_directory,./run_dir/tmp,./run_dir/tmp,./run_dir/tmp,./run_dir/tmp
method,similarity,similarity,similarity,similarity


In [62]:
par_df = run_parameters_df
key_name = 'method'
key_value = 'similarity'
similarity_df = get_key_value_df_II(par_df, key_name, key_value)
similarity_df

,BENCHMARK_1_GS_cos.yml,TEST_1_GS_cos.yml,BENCHMARK_1_GS_spearman.yml,TEST_1_GS_spearman.yml
spreadsheet_name_full_path,../data/spreadsheets/Hsap.nbs_OV.G.gene_som_mu...,../data/spreadsheets/TEST_1_gene_sample.tsv,../data/spreadsheets/Jinfeng_spreadsheet_sampl...,../data/spreadsheets/TEST_1_gene_sample.tsv
cluster_shared_volumn,not used,not used,not used,not used
results_directory,./run_dir/results,./run_dir/results,./run_dir/results,./run_dir/results
similarity_measure,cosine,cosine,spearman,spearman
signature_name_full_path,../data/spreadsheets/Hsap.nbs_UCEC.G.gene_som_...,../data/spreadsheets/TEST_1_signature.tsv,../data/spreadsheets/Jinfeng_spreadsheet_signa...,../data/spreadsheets/TEST_1_signature.tsv
rows_sampling_fraction,not used,not used,not used,not used
rwr_restart_probability,not used,not used,not used,not used
run_directory,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...,/Users/lanier4/git_clone/dlanier/Signature_Ana...
cols_sampling_fraction,not used,not used,not used,not used
processing_method,not used,not used,not used,not used


In [58]:
for ugg in similarity_df.index:
    if (similarity_df.loc[ugg].values == 'not used').any() == False:
        print('\n\t',ugg)



	 spreadsheet_name_full_path

	 results_directory

	 similarity_measure

	 signature_name_full_path

	 run_directory

	 tmp_directory

	 method
